<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
import joblib
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
import planetary_computer
from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, scale, calculate_band_indices, apply_mask, mask_land, mask_deeps, mask_elevation, all_masks, glcm_features, do_prediction

In [2]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "bootless"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-bootless-17072025


## Step 1.2: Configure STAC access and search parameters

In [3]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [4]:
# ## Use training data bounds

training = gpd.read_file("training-data/nm-bootless-bay-13072025_postcard.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

# TODO: configure colours...
# training.explore(column="observed")

In [5]:
datetime = "2024"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2024


In [6]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "count", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 5MB
Dimensions:      (y: 490, x: 262, time: 1)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    red          (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    blue         (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    green        (time, y, x) uint16 257kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    emad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    smad         (time, y, x) float32 514kB dask.array<chunksize=(1, 490, 262), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 257kB dask.arr

In [7]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [8]:
# Explore the site we are working on
# scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [9]:
scaled = calculate_band_indices(scaled)

In [10]:
## Moderate land mask
MNDWI_THRESHOLD = -0.2
mndwi_mask = scaled.mndwi > MNDWI_THRESHOLD

# mndwi_mask.odc.explore()

In [11]:
ln_bg = scaled['ln_bg'].values.flatten()  # flatten in case it's multi-dimensional
# Remove NaNs if present
ln_bg = ln_bg[~np.isnan(ln_bg)]
# Calculate mean and std
mean = ln_bg.mean()
std = ln_bg.std()


In [12]:
# Deep ocean mask
ln_bg_mask = scaled["ln_bg"] < 0

# ln_bg_mask.odc.explore()

In [13]:
e84_catalog = "https://earth-search.aws.element84.com/v1/"
e84_client = Client.open(e84_catalog)
collection = "cop-dem-glo-30"

items = e84_client.search(
    collections=[collection],
    bbox=bbox
).item_collection()

items

# Using geobox means it will load the elevation data the same shape as the other data
elevation = load(items, measurements=["data"], geobox=data.odc.geobox).squeeze()

# True where data is above 10m elevation
elevation_mask = elevation.data < 10

# elevation_mask.odc.explore()

In [14]:
masked = scaled.where(mndwi_mask)

In [15]:
# elevation mask is having issues
all_masks = mndwi_mask & ln_bg_mask & elevation_mask
all_masks = all_masks.rio.write_crs("EPSG:3832") # Use .rio.reproject

centroid = scaled.odc.geobox.geographic_extent.centroid.coords[0][::-1]
m = folium.Map(location=centroid, zoom_start=14)

scaled.odc.to_rgba(bands=["red", "green", "blue"], vmin=0, vmax=0.3).odc.add_to(m, name="RGB")
mndwi_mask.where(mndwi_mask == 0).odc.add_to(m, name="MNDWI Mask", vmin=0, vmax=1)
# ndti_mask.where(ndti_mask == 0).odc.add_to(m, name="NDTI Mask", vmin=0, vmax=1)
ln_bg_mask.where(ln_bg_mask == 0).odc.add_to(m, name="ln_bg Mask", vmin=0, vmax=1)
# all_masks.where(all_masks == 0).odc.add_to(m, name="All Masks", vmin=0, vmax=1)
all_masks.odc.add_to(m, name="All Masks", vmin=0, vmax=1) 

folium.LayerControl().add_to(m)

# m

In [16]:
# Now apply the mask, and view the masked region

masked = scaled.where(all_masks)
# masked.odc.explore(
#     vmin=0, vmax=0.3, bands=["red", "green", "blue"], name=f"{site}-masked", tiles=basemaps.OpenStreetMap.Mapnik  
# )

### GLCM texture analysis

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [17]:
WINDOW_SIZE = 9
LEVELS = 32

# Input
max = masked.blue.max().values
min = masked.blue.min().values
# Scale to 0-LEVELS for GLCM
img = ((masked.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)

# Extract overlapping windows
patches = sliding_window_view(img, (WINDOW_SIZE, WINDOW_SIZE))
# Shape: (rows, cols, win_y, win_x)


/tmp/ipykernel_1881/3068184428.py:8: RuntimeWarning: invalid value encountered in cast
  img = ((masked.blue - min) / (max - min) * (LEVELS - 1)).clip(0, LEVELS - 1).values.astype(np.uint8)


In [18]:
import numpy as np # Ensure numpy is imported if not already

# Assuming 'patches' is a 4D NumPy array with dimensions (y_coords, x_coords, window_y_size, window_x_size)
# To get the first patch (at y=0, x=0), you would index it like this:
sample_patch_data = patches[0, 0, :, :]

# Verify the shape of the extracted sample patch data
print(f"Shape of sample_patch_data: {sample_patch_data.shape}")

# Call glcm_features directly on this 2D sample data
sample_result = glcm_features(sample_patch_data)

# Print the shape of the result to get the number of features
print(f"Shape of glcm_features output for a single patch: {sample_result.shape}")

Shape of sample_patch_data: (9, 9)
Shape of glcm_features output for a single patch: (7,)


In [20]:
# Use apply_ufunc to vectorize over (row, col) dimensions
result = xr.apply_ufunc(
    glcm_features,
    xr.DataArray(patches, dims=["y", "x", "win_y", "win_x"]),
    input_core_dims=[["win_y", "win_x"]],
    output_core_dims=[["feature"]],
    vectorize=True,
    dask="parallelized",
    output_dtypes=[np.float32]
)

# Add coordinates & names
pad = WINDOW_SIZE - 1
result = result.assign_coords({
    "y": masked.y[: -pad],
    "x": masked.x[: -pad],
    "feature": ["contrast", "homogeneity", "energy", "ASM", "correlation", "mean", "entropy"]
})

result_bands = result.to_dataset(dim="feature")

# Combine with original
masked_plus = masked.copy()
masked_plus = masked_plus.assign(result_bands)
# masked_plus = masked_plus.drop_vars("count")
masked_plus


<xarray.Dataset> Size: 30MB
Dimensions:      (y: 490, x: 262)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    time         datetime64[ns] 8B 2024-01-01
Data variables: (12/34)
    nir          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    green        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    emad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    smad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    ...           ...
    homogeneity  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    energy       (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    ASM          (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    correlation  (y, x) float32 514kB 1.0 1.0 1.0 1.0 1.0 ... nan nan nan nan
    mean         (y, x) float32 514kB 0.0 0.0 0.0 0.0 0.0 ... nan nan nan nan
    entropy      (y, x) float32 514kB -1.443e-10 -1.443e-10 ... nan nan

In [21]:
print(len(masked_plus.data_vars), 'columns')
print('including', masked_plus.data_vars)

34 columns
including Data variables:
    nir          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    green        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    emad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    smad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    bcmad        (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    nir08        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    nir09        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    swir16       (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    swir22       (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    coastal      (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    reded

In [22]:

masked_plus.odc.explore(
    vmin=0, vmax=0.3, bands=["red", "green", "blue"], name=f"{site}-masked", tiles=basemaps.Esri.WorldImagery
)

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:55: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


In [24]:
# Re-apply the mask
masked_plus = masked_plus.where(all_masks)

masked_da = masked_plus.to_dataarray()

In [25]:
# Convert to a stacked array of observations
# stacked_arrays = stacked_arrays.squeeze()
masked_da = masked_da.squeeze()#.stack(dims=["y", "x"])#.transpose()
# stacked_arrays = stacked_arrays.to_dataarray()

In [26]:
stacked_arrays_2d = masked_da.stack(new_dim=("y", "x")) 
reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')
stacked_arrays_2d.shape


(34, 128380)

In [27]:
# Replace any infinities with NaN
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("inf"))
stacked_arrays_2d = stacked_arrays_2d.where(stacked_arrays_2d != float("-inf"))

# Replace any NaN values with 0
df = stacked_arrays_2d.squeeze().fillna(0).transpose().to_pandas()

# Remove the all-zero rows
zero_mask = (df == 0).all(axis=1)  # Creates a boolean Series
non_zero_df = df.loc[~zero_mask]  # Filters out all-zero rows

# Create a new array to hold the predictions
full_pred = pd.Series(np.nan, index=df.index)

reordered_data_array = stacked_arrays_2d.transpose('new_dim', 'variable')

## Model prediction

In [28]:
reordered_data_array

<xarray.DataArray (new_dim: 128380, variable: 34)> Size: 35MB
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
    time         datetime64[ns] 8B 2024-01-01
    spatial_ref  int64 8B 0
  * variable     (variable) object 272B 'nir' 'red' 'blue' ... 'mean' 'entropy'
  * new_dim      (new_dim) object 1MB MultiIndex
  * y            (new_dim) float64 1MB -1.055e+06 -1.055e+06 ... -1.06e+06
  * x            (new_dim) float64 1MB -3.036e+05 -3.036e+05 ... -3.01e+05

In [29]:
masked_plus

<xarray.Dataset> Size: 30MB
Dimensions:      (y: 490, x: 262)
Coordinates:
  * y            (y) float64 4kB -1.055e+06 -1.055e+06 ... -1.06e+06 -1.06e+06
  * x            (x) float64 2kB -3.036e+05 -3.036e+05 ... -3.01e+05 -3.01e+05
    time         datetime64[ns] 8B 2024-01-01
    spatial_ref  int64 8B 0
Data variables: (12/34)
    nir          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    red          (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    blue         (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    green        (y, x) float64 1MB nan nan nan nan nan ... nan nan nan nan nan
    emad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    smad         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    ...           ...
    homogeneity  (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    energy       (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    ASM          (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    correlation  (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    mean         (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan
    entropy      (y, x) float32 514kB nan nan nan nan nan ... nan nan nan nan

In [30]:
model = joblib.load("models/nm-17072025-test.model") 

In [31]:
## Predict the classes
# predicted = model.predict(reordered_data_array)

# Predict the classes
predicted = model.predict(reordered_data_array)

In [32]:
# Reshape back to the original 2D array
reordered_data_array = predicted.reshape(len(masked_plus.y), len(masked_plus.x))

# Convert to an xarray again, because it's easier to work with
predicted_da = xr.DataArray(
    reordered_data_array, coords={"y": masked_plus.y, "x": masked_plus.x}, dims=["y", "x"]
)

In [33]:
# postcard_params = postcard_df.columns[1:]
# print(postcard_params)

In [34]:
print(predicted_da.dtype)  # Check the dtype of your DataArray
predicted_da = predicted_da.astype('float32')  # Convert to float32

# Check for NaN values
if np.isnan(predicted_da).any():
    print("NaN values found in the data")
    # Handle NaN values, e.g. by filling them
    predicted_da = predicted_da.fillna(0)  # Replace NaN with 0 or appropriate value

float32


In [35]:
from matplotlib import colors

classes = [
    [1, "sediment", "#8c8c8c"],
    [2, "sand", "#fedd24"],
    [3, "rubble", "#f8ffb4"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#b9df6f"],
    [6, "coral", "#a011c3"],
    [7, "rock", "#804600"],
    [8, "deeps", "#011b61"],
    [9, "mangrove", "#086a39"],
    [10, "land", "#ffffff"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

# predicted_da.plot.imshow(cmap=c_map, norm=norm, size=10)
predicted_da.odc.explore(cmap=c_map, tiles=basemaps.Esri.WorldImagery)

In [36]:
from matplotlib import colors

classes = [
    [1, "sediment", "#00000000"],
    [2, "sand", "#00000000"],
    [3, "rubble", "#00000000"],
    [4, "seagrass", "#6df7dc"],
    [5, "seaweed", "#00000000"],
    [6, "coral", "#00000000"],
    [7, "rock", "#00000000"],
    [8, "deeps", "#00000000"],
    [9, "mangrove", "#00000000"],
    [10, "land", "#00000000"],
]

values_list = [c[0] for c in classes]
color_list = [c[2] for c in classes]

# Build a listed colormap.
c_map_seagrass = colors.ListedColormap(color_list)
bounds = values_list + [14]
norm = colors.BoundaryNorm(bounds, c_map.N)

predicted_da.odc.explore(cmap=c_map_seagrass, tiles=basemaps.Esri.WorldImagery)
# predicted_da.plot.imshow(cmap=c_map_seagrass, norm=norm, size=10)

In [ ]:
predicted_da.odc.write_cog(f"{version}.tiff", overwrite=True)